<a href="https://colab.research.google.com/github/jeduardosleite/heart_attack_analysis/blob/main/anotacoes_mod5_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving aula_1.csv to aula_1.csv
Saving aula_2.csv to aula_2.csv


In [2]:
query_1 = pd.read_csv('aula_1.csv')
query_2 = pd.read_csv('aula_2.csv')

# DATA ANALYSIS: HEART ATTACK


###**Objective 1:**
Understand the distribution of the variable 'sex' with the chance of heart attack.

- ***sex 0*** = female
- ***sex 1*** = male
- ***output 0*** = small chance of heart attack
- ***output 1*** = high chance of heart attack

This is exposed in query 1.

###**Objective 2:**
Analyze the maximum, minimum and average age in the 'sex - output' relationship.

This is exposed in query 2.

----

First I wrote the following code:
```sql
CREATE EXTERNAL TABLE heartattack (
  age INT,
  sex INT,
  cp INT,
  trtbps INT,
  chol INT,
  fbs INT,
  restecg INT,
  thatach INT,
  exng INT,
  oldpeak DOUBLE,
  slp INT,
  caa INT,
  thall INT,
  output INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
LOCATION 's3://mod5-bucket-heartattack/';
```



##Objective 1

### **Query 1**
```sql
SELECT
    COUNT(sex) AS Amount,
    sex AS "Sex",
    "output" AS heart_attack_chance,
    CASE
       WHEN sex = 0 AND "output" = 0 THEN 'Female with less chance'
       WHEN sex = 1 AND "output" = 0 THEN 'Male with less chance'
       WHEN sex = 0 AND "output" = 1 THEN 'Female with more chance'
       WHEN sex = 1 AND "output" = 1 THEN 'Male with more chance'
    END AS sexo_describe
FROM heartattack
GROUP BY "output", sex
ORDER BY Amount DESC;
```

I concluded that males have a greater chance of heart attack.

In [3]:
query_1

,Amount,Sex,heart_attack_chance,sexo_describe
0,114,1,0,Male with less chance
1,93,1,1,Male with more chance
2,72,0,1,Female with more chance
3,24,0,0,Female with less chance


## Objective 2

### Query 2

```sql
SELECT
    MAX(age) AS max_age,
    MIN(age) AS min_age,
    ROUND(AVG(age)) AS median_age,
    CASE
        WHEN "output" = 1 THEN 'More chance'
        ELSE 'Less chance'
    END AS heart_attack_chance,
    CASE
        WHEN sex = 1 THEN 'Male'
        ELSE 'Female'
    END AS Sex
FROM heartattack
GROUP BY sex, "output";
```

Men are more likely to have a heart attack at an earlier age as their average age is younger than women.

The maximum and minimum age corroborate this conclusion.
- Males have a greater chance of having a heart attack at an earlier age (minimum age 29)
- There is a discrepant difference in the minimum age. Females have a 5-year advantage in the greater chance of heart attack.
When we talk about less chance, the difference becomes even greater, with an 8-year advantage.

In [4]:
query_2

,max_age,min_age,median_age,heart_attack_chance,Sex
0,70,29,51.0,More chance,Male
1,76,34,55.0,More chance,Female
2,77,35,56.0,Less chance,Male
3,66,43,59.0,Less chance,Female
